# Set up

In [1]:
import os
import json
import yaml
import shutil
import zipfile
import random
import numpy as np
import pandas as pd

In [2]:
os.getcwd()

'/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/code'

In [3]:
!ls ../data_atl

img_atl_2022  img_middown_0206	img_middown_0208_detected  inferred
img_cs_0204   img_middown_0208	inference_log		   seg


In [4]:
base_dir = '../data_atl'

# Import labels

## import log_detected_df

In [5]:
base_dir = '../data_atl'
log_detected_df = pd.read_csv(os.path.join(base_dir, "inferred", "log_detected_df.csv")).drop(columns = ["Unnamed: 0"])
log_detected_df = log_detected_df.sort_values(by="saved_filename")
log_detected_df

,range_index,file_id,detection_index,class_id,xmin,ymin,xmax,ymax,confidence,area,saved_filename
185,33,100__15__right,1,4,0,0,15,42,0.11,630,100__15__right__1__4__0.11__630.jpg
214,37,100__17__right,0,0,593,293,608,322,0.51,435,100__17__right__0__0__0.51__435.jpg
217,38,101__0__right,0,3,349,291,377,336,0.25,1260,101__0__right__0__3__0.25__1260.jpg
404,77,102__3__left,0,3,238,281,257,303,0.47,418,102__3__left__0__3__0.47__418.jpg
325,58,103__8__left,0,3,144,306,161,331,0.26,425,103__8__left__0__3__0.26__425.jpg
...,...,...,...,...,...,...,...,...,...,...,...
161,30,94__10__left,1,1,604,339,624,363,0.12,480,94__10__left__1__1__0.12__480.jpg
87,16,94__17__right,0,3,606,267,640,334,0.21,2278,94__17__right__0__3__0.21__2278.jpg
42,7,94__2__right,0,3,217,297,238,328,0.40,651,94__2__right__0__3__0.4__651.jpg
43,7,94__2__right,1,1,217,297,237,328,0.25,620,94__2__right__1__1__0.25__620.jpg


In [6]:
class_id_nm = pd.DataFrame({
    "class_id": [0, 1, 2, 3, 4],
    "class_nm": ["prohibAnyT", "prohibSomeT", "prohibBusStop", "prohibClean", "allow"]
})
class_id_nm

log_detected_df = log_detected_df.merge(class_id_nm, on="class_id", how="left")
log_detected_df

,range_index,file_id,detection_index,class_id,xmin,ymin,xmax,ymax,confidence,area,saved_filename,class_nm
0,33,100__15__right,1,4,0,0,15,42,0.11,630,100__15__right__1__4__0.11__630.jpg,allow
1,37,100__17__right,0,0,593,293,608,322,0.51,435,100__17__right__0__0__0.51__435.jpg,prohibAnyT
2,38,101__0__right,0,3,349,291,377,336,0.25,1260,101__0__right__0__3__0.25__1260.jpg,prohibClean
3,77,102__3__left,0,3,238,281,257,303,0.47,418,102__3__left__0__3__0.47__418.jpg,prohibClean
4,58,103__8__left,0,3,144,306,161,331,0.26,425,103__8__left__0__3__0.26__425.jpg,prohibClean
...,...,...,...,...,...,...,...,...,...,...,...,...
401,30,94__10__left,1,1,604,339,624,363,0.12,480,94__10__left__1__1__0.12__480.jpg,prohibSomeT
402,16,94__17__right,0,3,606,267,640,334,0.21,2278,94__17__right__0__3__0.21__2278.jpg,prohibClean
403,7,94__2__right,0,3,217,297,238,328,0.40,651,94__2__right__0__3__0.4__651.jpg,prohibClean
404,7,94__2__right,1,1,217,297,237,328,0.25,620,94__2__right__1__1__0.25__620.jpg,prohibSomeT


## make img_middown_0208_detected

In [7]:
base_dir = '../data_atl'
folder_path = os.path.join(base_dir, "img_middown_0208")
print(folder_path)
print()
filenames = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) & f.endswith((".jpg", ".png"))]
filenames = sorted(filenames)
filenames = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) & f.endswith((".jpg", ".png"))]
print(len(filenames))
filenames[:5]
# {edge_id}__{point_id}__{side}__{j}__{class_id}__{confidence}__{area}

../data_atl/img_middown_0208

7788


['../data_atl/img_middown_0208/gsv__215__12__right__139.361__2019-10__33.775806801,-84.379175182.jpg',
 '../data_atl/img_middown_0208/gsv__173__11__right__48.943__2022-03__33.785063793,-84.384255698.jpg',
 '../data_atl/img_middown_0208/gsv__530__3__left__78.461__2024-07__33.750691195,-84.393234343.jpg',
 '../data_atl/img_middown_0208/gsv__239__4__right__139.456__2024-11__33.796786698,-84.38753574.jpg',
 '../data_atl/img_middown_0208/gsv__405__9__right__168.371__2024-06__33.752104796,-84.395139153.jpg']

In [8]:
unique_detected_ids = log_detected_df["saved_filename"].tolist()
unique_detected_ids = [f.split("__") for f in unique_detected_ids] 
unique_detected_ids = ["__".join([f[0], f[1], f[2]]) for f in unique_detected_ids] 
print(len(unique_detected_ids))
unique_detected_ids[:5]

406


['100__15__right',
 '100__17__right',
 '101__0__right',
 '102__3__left',
 '103__8__left']

In [9]:
filename_of_detected = list()

for filename in filenames:
    temp = filename.split("/")[-1].split("__")
    temp = "__".join([temp[1], temp[2], temp[3]]) 
    #print(temp)
    if temp in unique_detected_ids:
        filename_of_detected.append(filename)
filename_of_detected = [f.split("/")[-1] for f in filename_of_detected]
print(len(filename_of_detected))
filename_of_detected[:5]

357


['gsv__413__12__right__nan__2024-07__33.751050768,-84.393232338.jpg',
 'gsv__503__13__left__-56.362__2024-11__33.757213141,-84.381455751.jpg',
 'gsv__468__5__left__135.041__2024-07__33.754496624,-84.387807485.jpg',
 'gsv__390__9__left__78.267__2024-07__33.750005066,-84.391331876.jpg',
 'gsv__76__4__left__140.012__2023-01__33.772343059,-84.387770359.jpg']

In [10]:
base_dir = '../data_atl'
dst = os.path.join(base_dir, "img_middown_0208_detected")
os.makedirs(dst, exist_ok=True)

for filename in filename_of_detected:
    src = os.path.join(base_dir, "img_middown_0208", filename) 
    dst = os.path.join(base_dir, "img_middown_0208_detected", filename)
    shutil.copy(src, dst)

In [11]:
base_dir = '../data_atl'
folder_path = os.path.join(base_dir, "img_middown_0208_detected")
print(folder_path)
print()
filenames = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) & f.endswith((".jpg", ".png"))]
filenames = sorted(filenames)
filenames = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) & f.endswith((".jpg", ".png"))]
print(len(filenames))
filenames[:5]
# {edge_id}__{point_id}__{side}__{j}__{class_id}__{confidence}__{area}

../data_atl/img_middown_0208_detected

357


['../data_atl/img_middown_0208_detected/gsv__413__12__right__nan__2024-07__33.751050768,-84.393232338.jpg',
 '../data_atl/img_middown_0208_detected/gsv__503__13__left__-56.362__2024-11__33.757213141,-84.381455751.jpg',
 '../data_atl/img_middown_0208_detected/gsv__468__5__left__135.041__2024-07__33.754496624,-84.387807485.jpg',
 '../data_atl/img_middown_0208_detected/gsv__390__9__left__78.267__2024-07__33.750005066,-84.391331876.jpg',
 '../data_atl/img_middown_0208_detected/gsv__76__4__left__140.012__2023-01__33.772343059,-84.387770359.jpg']

## add original img filename to log_detected_df

In [12]:
filename_of_detected = list()
filename_of_detected_short = list()
for filename in filenames:
    temp = filename.split("/")[-1].split("__")
    temp = "__".join([temp[1], temp[2], temp[3]]) 
    #print(temp)
    filename_of_detected.append(filename)
    filename_of_detected_short.append(temp)
filename_of_detected = [f.split("/")[-1] for f in filename_of_detected]
print(len(filename_of_detected))
print(filename_of_detected[:5])
print(len(filename_of_detected_short))
print(filename_of_detected_short[:5])

357
['gsv__413__12__right__nan__2024-07__33.751050768,-84.393232338.jpg', 'gsv__503__13__left__-56.362__2024-11__33.757213141,-84.381455751.jpg', 'gsv__468__5__left__135.041__2024-07__33.754496624,-84.387807485.jpg', 'gsv__390__9__left__78.267__2024-07__33.750005066,-84.391331876.jpg', 'gsv__76__4__left__140.012__2023-01__33.772343059,-84.387770359.jpg']
357
['413__12__right', '503__13__left', '468__5__left', '390__9__left', '76__4__left']


In [13]:
filename_of_detected_df = pd.DataFrame({
    "filename_of_detected": filename_of_detected,
    "file_id": filename_of_detected_short
}
)
filename_of_detected_df

,filename_of_detected,file_id
0,gsv__413__12__right__nan__2024-07__33.75105076...,413__12__right
1,gsv__503__13__left__-56.362__2024-11__33.75721...,503__13__left
2,gsv__468__5__left__135.041__2024-07__33.754496...,468__5__left
3,gsv__390__9__left__78.267__2024-07__33.7500050...,390__9__left
4,gsv__76__4__left__140.012__2023-01__33.7723430...,76__4__left
...,...,...
352,gsv__93__16__right__45.919__2023-01__33.774237...,93__16__right
353,gsv__117__0__left__-130.377__2023-01__33.77471...,117__0__left
354,gsv__208__20__left__nan__2024-07__33.774780359...,208__20__left
355,gsv__102__3__left__-45.135__2022-02__33.778507...,102__3__left


In [14]:
log_detected_df = log_detected_df.merge(filename_of_detected_df, on="file_id", how="left")
log_detected_df

,range_index,file_id,detection_index,class_id,xmin,ymin,xmax,ymax,confidence,area,saved_filename,class_nm,filename_of_detected
0,33,100__15__right,1,4,0,0,15,42,0.11,630,100__15__right__1__4__0.11__630.jpg,allow,gsv__100__15__right__311.014__2024-07__33.7787...
1,37,100__17__right,0,0,593,293,608,322,0.51,435,100__17__right__0__0__0.51__435.jpg,prohibAnyT,gsv__100__17__right__319.704__2024-07__33.7786...
2,38,101__0__right,0,3,349,291,377,336,0.25,1260,101__0__right__0__3__0.25__1260.jpg,prohibClean,gsv__101__0__right__101.282__2022-02__33.77832...
3,77,102__3__left,0,3,238,281,257,303,0.47,418,102__3__left__0__3__0.47__418.jpg,prohibClean,gsv__102__3__left__-45.135__2022-02__33.778507...
4,58,103__8__left,0,3,144,306,161,331,0.26,425,103__8__left__0__3__0.26__425.jpg,prohibClean,gsv__103__8__left__-40.348__2019-10__33.776024...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,30,94__10__left,1,1,604,339,624,363,0.12,480,94__10__left__1__1__0.12__480.jpg,prohibSomeT,gsv__94__10__left__221.508__2022-02__33.774833...
402,16,94__17__right,0,3,606,267,640,334,0.21,2278,94__17__right__0__3__0.21__2278.jpg,prohibClean,gsv__94__17__right__403.747__2022-02__33.77509...
403,7,94__2__right,0,3,217,297,238,328,0.40,651,94__2__right__0__3__0.4__651.jpg,prohibClean,gsv__94__2__right__400.641__2022-02__33.774533...
404,7,94__2__right,1,1,217,297,237,328,0.25,620,94__2__right__1__1__0.25__620.jpg,prohibSomeT,gsv__94__2__right__400.641__2022-02__33.774533...


# Build json file 

In [16]:
# Create the COCO dictionary
coco_format = {
    "images": [],
    "annotations": [],
    "categories": []
}

### categories ###
# Extract unique class names and assign category IDs
class_names = log_detected_df["class_nm"].unique()
new_order = ['prohibAnyT', 'prohibSomeT', 'prohibBusStop','prohibClean', 'allow']
class_names = np.array(new_order)
category_id_map = {name: i for i, name in enumerate(class_names, 1)} # very important to start from 1 not 0 for the category_id

# Add category information
for class_name, class_id in category_id_map.items():
    coco_format["categories"].append({
        "id": class_id,
        "name": class_name,
        #"supercategory": "sign"
    })
print(coco_format["categories"])
print() 



### images ###
# Extract unique image files and assign image IDs
image_files = log_detected_df["filename_of_detected"].unique()
image_id_map = {file: i for i, file in enumerate(image_files)}
print(len(image_id_map))
print() 

# Add image metadata
for file_id in image_id_map:
    image_rows = log_detected_df[log_detected_df["filename_of_detected"] == file_id]
    #print(image_rows)
    # Assuming all detections for the same file share the same dimensions
    coco_format["images"].append({
        "id": image_id_map[file_id],
        "file_name": str(image_rows["filename_of_detected"].iloc[0]),  # very important
        "width": 640,
        "height": 640
    })
print(len(coco_format["images"]))
print() 



### annotations ###
# Add annotation data
annotation_id = 0
for _, row in log_detected_df.iterrows():
    coco_format["annotations"].append({
        "id": annotation_id,
        "image_id": image_id_map[row["filename_of_detected"]],   # HERE
        "category_id": category_id_map[row["class_nm"]],
        "bbox": [row["xmin"], row["ymin"], row["xmax"] - row["xmin"], row["ymax"] - row["ymin"]],
        "area": row["area"],
        "iscrowd": 0,
        #"confidence": row["confidence"]
        "segmentation": []
    })
    annotation_id += 1
print(len(coco_format["annotations"]))
print() 

[{'id': 1, 'name': 'prohibAnyT'}, {'id': 2, 'name': 'prohibSomeT'}, {'id': 3, 'name': 'prohibBusStop'}, {'id': 4, 'name': 'prohibClean'}, {'id': 5, 'name': 'allow'}]

357

357

406



In [24]:
coco_format

{'images': [{'id': 0,
   'file_name': 'gsv__100__15__right__311.014__2024-07__33.778700264,-84.386663806.jpg',
   'width': 640,
   'height': 640},
  {'id': 1,
   'file_name': 'gsv__100__17__right__319.704__2024-07__33.778697074,-84.386718894.jpg',
   'width': 640,
   'height': 640},
  {'id': 2,
   'file_name': 'gsv__101__0__right__101.282__2022-02__33.778323807,-84.37504695.jpg',
   'width': 640,
   'height': 640},
  {'id': 3,
   'file_name': 'gsv__102__3__left__-45.135__2022-02__33.778507826,-84.373641481.jpg',
   'width': 640,
   'height': 640},
  {'id': 4,
   'file_name': 'gsv__103__8__left__-40.348__2019-10__33.776024203,-84.382590789.jpg',
   'width': 640,
   'height': 640},
  {'id': 5,
   'file_name': 'gsv__106__14__left__-40.277__2024-07__33.776196542,-84.385134323.jpg',
   'width': 640,
   'height': 640},
  {'id': 6,
   'file_name': 'gsv__109__0__left__-44.798__2023-01__33.775332305,-84.377498341.jpg',
   'width': 640,
   'height': 640},
  {'id': 7,
   'file_name': 'gsv__109__0

In [17]:
coco_format["categories"]

[{'id': 1, 'name': 'prohibAnyT'},
 {'id': 2, 'name': 'prohibSomeT'},
 {'id': 3, 'name': 'prohibBusStop'},
 {'id': 4, 'name': 'prohibClean'},
 {'id': 5, 'name': 'allow'}]

In [22]:
len(coco_format["images"])

357

In [23]:
len(coco_format["annotations"])

406

## export

In [20]:
base_dir = '../data_atl'
output_path = os.path.join(base_dir, "inferred", "inferred_model_nc_5.json")

In [21]:
# Save COCO JSON to file
with open(output_path, "w") as f:
    json.dump(coco_format, f, indent=4
              #, default=convert_numpy
             )

print(f"COCO annotations saved to {output_path}!")

COCO annotations saved to ../data_atl/inferred/inferred_model_nc_5.json!


In [ ]:
# Save COCO JSON to file
with open(output_path, "w") as f:
    json.dump(coco_data, f, indent=4)

print(f"COCO annotations saved to {output_path}!")

# Preload known annotations 